# Introduction to WaterTAP Costing

# DELETE THIS CELL BEFORE MERGING
Outline:
- Just build an RO flowsheet
- Walk through adding the existing costing for each unit model (consider showing snippets of the unit model costing files or documentation with our explanations)
- Walk through how to modify the existing parameters
- How to add system level metrics (LCOW, SEC, etc.)
- Walk through how to go from simulating costing to optimizing it (add objective function for minimizing LCOW)
- Add a try it yourself for optimizing for a different objective (SEC)
- Walk through how they might add their own costing (register their own flow types) as shown in #2 here: https://watertap.readthedocs.io/en/latest/technical_reference/costing/costing_base.html

This tutorial will demonstrate how to add costing for a reverse osmosis unit model as well as how to change the default parameter values and add custom costing relationships.

## Step 1: Import Modules

In [47]:
# Imports from Pyomo
from pyomo.environ import (
    ConcreteModel,
    Var,
    Constraint,
    Objective,
    value,
    assert_optimal_termination,
    TransformationFactory,
    units as pyunits,
)

# Imports from IDAES
from idaes.core import FlowsheetBlock
from idaes.core import UnitModelCostingBlock

# Imports from WaterTAP
from watertap.property_models.seawater_prop_pack import SeawaterParameterBlock
from watertap.unit_models.pressure_changer import Pump
from watertap.unit_models.reverse_osmosis_0D import (
    ReverseOsmosis0D,
    ConcentrationPolarizationType,
    MassTransferCoefficient,
    PressureChangeType,
)
from watertap.costing import WaterTAPCosting
from watertap.costing.unit_models.reverse_osmosis import (
    cost_high_pressure_reverse_osmosis,
)
from watertap.costing.unit_models.pump import cost_high_pressure_pump

## Step 2: Create an RO Model

In [48]:
# Build flowsheet essentials
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)

# Build property model
m.fs.properties = SeawaterParameterBlock()

# Build unit model
m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    has_pressure_change=True,
    pressure_change_type=PressureChangeType.calculated,
    mass_transfer_coefficient=MassTransferCoefficient.calculated,
    concentration_polarization_type=ConcentrationPolarizationType.calculated,
)

## Step 3: Add RO Costing

### Step 3a: Create the Costing Model

`WaterTAPCosting` contain extensions, methods, and variables and constraints common to all WaterTAP Costing Packages. In other words, this sets up the base cost model, which we will expand upon by adding RO-specific cost terms. This is conventionally defined as `m.fs.costing`, and we'll also establish the base currency for the cost model.

In [49]:
# Set up WaterTAP Costing
m.fs.costing = WaterTAPCosting()

# Set units of base currency
m.fs.costing.base_currency = pyunits.USD_2020

WaterTAP Costing is the costing block added at the flowsheet level. WaterTAP costing aggregates the unit model specific cost terms for all the unit models in the flowsheet.

In [50]:
m.fs.costing.display()

Block fs.costing

  Variables:
    total_investment_factor : Total investment factor [investment cost/equipment cost]
        Size=1, Index=None, Units=dimensionless
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :   1.0 :  None :  True : False :  Reals
    maintenance_labor_chemical_factor : Maintenance-labor-chemical factor [fraction of equipment cost/year]
        Size=1, Index=None, Units=1/a
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  0.03 :  None :  True : False :  Reals
    utilization_factor : Plant capacity utilization [fraction of uptime]
        Size=1, Index=None, Units=dimensionless
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :   0.9 :  None :  True : False :  Reals
    electricity_cost : Electricity cost
        Size=1, Index=None, Units=USD_2018/kWh
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  0.07 :  None :  True :

### Step 3b: Add RO Unit Model Costing Block
Next, add the `UnitModelCostingBlock` for RO, which will set up all the RO-specific variables and constraints related to costing the unit. There are two configuration arguments on `UnitModelCostingBlock` that must be set. The first is `flowsheet_costing_block`, which refers to the instance of WaterTAP costing (usually `m.fs.costing`). The second argument is `costing_method`, which allows the user to choose between different cost assumptions. As shown  below, RO has two costing methods:

<p align="center">
<img src='graphics/ro_costing_doc.png' alt="RO Costing Assumptions" width="1000">
</p>

The UnitModelCostingBlock adds the unit model cost block to the flowsheet level costing block

In [ ]:
# Create RO costing block
m.fs.RO.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
)

## Step 4: Display the cost model

In [52]:
# Print the cost model for RO
m.fs.costing.reverse_osmosis.display()

Block fs.costing.reverse_osmosis

  Variables:
    factor_membrane_replacement : Membrane replacement factor [fraction of membrane replaced/year]
        Size=1, Index=None, Units=1/a
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :   0.2 :  None :  True : False :  Reals
    membrane_cost : Membrane cost
        Size=1, Index=None, Units=USD_2018/m**2
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :    30 :  None :  True : False :  Reals
    high_pressure_membrane_cost : Membrane cost
        Size=1, Index=None, Units=USD_2018/m**2
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :    75 :  None :  True : False :  Reals

  Objectives:
    None

  Constraints:
    None


In [64]:
m.fs.RO.costing.display()

Block fs.RO.costing

  Variables:
    capital_cost : Unit capital cost
        Size=1, Index=None, Units=USD_2020
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :     0 : 1186.2709335101972 :  None : False : False : NonNegativeReals
    fixed_operating_cost : Unit fixed operating cost
        Size=1, Index=None, Units=USD_2020/a
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :     0 : 118.62709335102409 :  None : False : False : NonNegativeReals

  Objectives:
    None

  Constraints:
    capital_cost_constraint : Size=1
        Key  : Lower : Body                    : Upper
        None :   0.0 : -2.2737367544323206e-13 :   0.0
    fixed_operating_cost_constraint : Size=1
        Key  : Lower : Body                  : Upper
        None :   0.0 : 4.348521542851813e-12 :   0.0


## Step 5: Modify Existing Parameters

Let's say we want to change the membrane unit cost in the conventional case from 75 USD/m2 to 60 USD/m2. The naming convention for updating these parameters is `m.fs.costing.CostingMethod.Parameter.fix(new_value)`, where `CostingMethod` is the `costing_method` argument without the "cost" prefix.

In [53]:
# Update RO membrane cost
m.fs.costing.reverse_osmosis.high_pressure_membrane_cost.fix(60)

# Print the cost model for RO
m.fs.costing.reverse_osmosis.display()

Block fs.costing.reverse_osmosis

  Variables:
    factor_membrane_replacement : Membrane replacement factor [fraction of membrane replaced/year]
        Size=1, Index=None, Units=1/a
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :   0.2 :  None :  True : False :  Reals
    membrane_cost : Membrane cost
        Size=1, Index=None, Units=USD_2018/m**2
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :    30 :  None :  True : False :  Reals
    high_pressure_membrane_cost : Membrane cost
        Size=1, Index=None, Units=USD_2018/m**2
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :    60 :  None :  True : False :  Reals

  Objectives:
    None

  Constraints:
    None


## Try It Yourself

Add a `UnitModelCostingBlock` for a high-pressure pump and change the cost from 1.908 USD/W to 1 USD/W given the following information.
<p align="center">
<img src='graphics/pump_costing_doc.png' alt="Pump Costing Assumptions" width="1000">
</p>

In [55]:
# Build pump unit model
m.fs.pump = Pump(
    property_package=m.fs.properties,
)

# Add pump UnitModelCostingBlock
m.fs.pump.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
    costing_method=cost_high_pressure_pump,
)

# Print the cost model for the pump before updating
m.fs.costing.high_pressure_pump.display()

# Update pump cost parameter
m.fs.costing.high_pressure_pump.cost.fix(1)

# Print the cost model for the pump to verify changes
m.fs.costing.high_pressure_pump.display()

Block fs.costing.high_pressure_pump

  Variables:
    cost : High pressure pump cost
        Size=1, Index=None, Units=USD_2018/W
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 1.908 :  None :  True : False :  Reals

  Objectives:
    None

  Constraints:
    None
Block fs.costing.high_pressure_pump

  Variables:
    cost : High pressure pump cost
        Size=1, Index=None, Units=USD_2018/W
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :     1 :  None :  True : False :  Reals

  Objectives:
    None

  Constraints:
    None


## Step 6: Add System Level Cost Metrics

The cost process function creates the variables that are used for calculating system level cost metrics:
- aggregate_capital_cost
- aggregate_fixed_operating_cost
- aggregate_variable_operating_cost
- aggregate_flow_costs
- total_capital_cost
- total_operating_cost

In [56]:
m.fs.costing.cost_process()

m.fs.costing.initialize()
# Add levelized cost of water
m.fs.costing.add_LCOW(m.fs.RO.mixed_permeate[0].flow_vol_phase['Liq'])
# Add specific energy consumption (SEC)
m.fs.costing.add_specific_energy_consumption(
    m.fs.RO.mixed_permeate[0].flow_vol_phase['Liq'],
)

# Display costing results
print("\nCosting Results:")
print(f"Total Capital Cost: {value(m.fs.costing.total_capital_cost):.2f} USD/year")
print(f"Total Operating Cost: {value(m.fs.costing.total_operating_cost):.2f} USD/year")
print(f"Levelized Cost of Water (LCOW): {value(m.fs.costing.LCOW):.2f} USD/m3")
print(f"Specific Energy Consumption (SEC): {value(m.fs.costing.specific_energy_consumption):.2f} kWh/m3")


Costing Results:
Total Capital Cost: 1186.27 USD/year
Total Operating Cost: 154.22 USD/year
Levelized Cost of Water (LCOW): 0.00 USD/m3
Specific Energy Consumption (SEC): 0.00 kWh/m3


## Try it Yourself

In [63]:
# Display the capital cost for pump 
m.fs.pump.costing.capital_cost.display()

# Display the capital cost for pump 
m.fs.RO.costing.capital_cost.display()

capital_cost : Unit capital cost
    Size=1, Index=None, Units=USD_2020
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 :   0.0 :  None : False : False : NonNegativeReals
capital_cost : Unit capital cost
    Size=1, Index=None, Units=USD_2020
    Key  : Lower : Value              : Upper : Fixed : Stale : Domain
    None :     0 : 1186.2709335101972 :  None : False : False : NonNegativeReals


## Step 7: Register Custom Flows

Could walk through how they might add their own costing (register their own flow types) as shown in #2 here: https://watertap.readthedocs.io/en/latest/technical_reference/costing/costing_base.html, but after some further reflection, I think this is best left for a post-session tutorial. This tutorial functions as a good introduction as it currently stands.